# ResNet

> Neural net model

In [ ]:
#| default_exp models.resnet

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn as nn

import torch
from torchinfo import summary


from omegaconf import OmegaConf
from hydra.utils import instantiate


from nimrod.models.conv import ConvLayer
from nimrod.models.core import Classifier
from nimrod.utils import get_device, set_seed

from typing import List, Optional, Callable, Any, Type
import logging
from functools import partial


Seed set to 42


In [ ]:
#| export
logger = logging.getLogger(__name__)
set_seed()

Seed set to 42


## Res Block

In [ ]:
#| export 
class ResBlock(nn.Module):
    def __init__(
            self,
            in_channels:int, # Number of input channels
            out_channels:int, # Number of output channels
            stride:int=1,
            kernel_size:int=3,
            activation:Optional[Type[nn.Module]]=nn.ReLU
        ):

        super().__init__()
        self.activation = activation()
        conv_block = []
        conv_ = partial(ConvLayer, stride=1, activation=activation, normalization=nn.BatchNorm2d)
        # conv stride 1 to be able to go deeper while keeping the same spatial resolution
        c1 = conv_(in_channels, out_channels, stride=1, kernel_size=kernel_size)
        # conv stride to be able to go wider in number of channels
        # activation will be added at very end
        c2 = conv_(out_channels, out_channels, stride=stride, activation=None, kernel_size=kernel_size) #adding activation to the whole layer at the end c.f. forward
        conv_block += [c1,c2]
        self.conv_layer = nn.Sequential(*conv_block)

        if in_channels == out_channels:
            self.id = nn.Identity()
        else:
            # resize x to match channels
            self.id = conv_(in_channels, out_channels, kernel_size=1, stride=1, activation=None)
        
        if stride == 1:
            self.pooling = nn.Identity()
        else:
            # resize x to match the stride
            self.pooling = nn.AvgPool2d(stride, ceil_mode=True)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.activation(self.conv_layer(x) + self.id(self.pooling(x)))

### Usage

In [ ]:
model = ResBlock(3, 8, stride=2)
x = torch.randn(1, 3, 32, 32)
y = model(x)
print(y.shape)
summary(model=model, input_size=(1, 3, 32, 32), depth=2)

torch.Size([1, 8, 16, 16])


Layer (type:depth-idx)                   Output Shape              Param #
ResBlock                                 [1, 8, 16, 16]            --
├─Sequential: 1-1                        [1, 8, 16, 16]            --
│    └─ConvLayer: 2-1                    [1, 8, 32, 32]            232
│    └─ConvLayer: 2-2                    [1, 8, 16, 16]            592
├─AvgPool2d: 1-2                         [1, 3, 16, 16]            --
├─ConvLayer: 1-3                         [1, 8, 16, 16]            --
│    └─Sequential: 2-3                   [1, 8, 16, 16]            40
├─ReLU: 1-4                              [1, 8, 16, 16]            --
Total params: 864
Trainable params: 864
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.37
Input size (MB): 0.01
Forward/backward pass size (MB): 0.20
Params size (MB): 0.00
Estimated Total Size (MB): 0.21

## ResNet

In [ ]:
#| export
class ResNet(nn.Module):
    def __init__(
            self,
            n_features: List[int]=[1, 8, 16, 32, 64, 32], # Number of input & output channels
            num_classes: int=10, # Number of classes
        ):

        super().__init__()
        logger.info("ResNet: init")
        layers = []
        res_ = partial(ResBlock, stride=2)

        layers.append(res_(in_channels=n_features[0], out_channels=n_features[1], stride=1))

        for i in range(1, len(n_features)-1):
            layers += [res_(in_channels=n_features[i], out_channels=n_features[i+1])]

        # last layer back to n_classes and flatten
        layers.append(res_(in_channels=n_features[-1], out_channels=num_classes))
        layers.append(nn.Flatten())

        # layers += [nn.Flatten(), nn.Linear(n_features[-1], num_classes, bias=False), nn.BatchNorm1d(num_classes)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

In [ ]:
x = torch.randn(64, 3, 28, 28)
model = ResNet(n_features=[3, 8, 16, 32, 64, 32], num_classes=10)
y = model(x)
print(y.shape)
summary(model=model, input_size=(64, 3, 28, 28), depth=2)

Seed set to 42
/Users/slegroux/miniforge3/envs/nimrod/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Seed set to 42
[15:31:10] INFO - ResNet: init


torch.Size([64, 10])


Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [64, 10]                  --
├─Sequential: 1-1                                  [64, 10]                  --
│    └─ResBlock: 2-1                               [64, 8, 28, 28]           864
│    └─ResBlock: 2-2                               [64, 16, 14, 14]          3,680
│    └─ResBlock: 2-3                               [64, 32, 7, 7]            14,528
│    └─ResBlock: 2-4                               [64, 64, 4, 4]            57,728
│    └─ResBlock: 2-5                               [64, 32, 2, 2]            29,888
│    └─ResBlock: 2-6                               [64, 10, 1, 1]            4,160
│    └─Flatten: 2-7                                [64, 10]                  --
Total params: 110,848
Trainable params: 110,848
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 197.51
Input size (MB): 0.60
Forward/backward pass size (MB): 37.29
P

## ResNetX


In [ ]:
#| export

class ResNetX(Classifier):
    def __init__(
        self,
        nnet:ResNet,
        num_classes:int,
        optimizer:Callable[...,torch.optim.Optimizer], # optimizer,
        scheduler: Optional[Callable[...,Any]]=None, # scheduler
        ):
        
        logger.info("ResNetX: init")
        super().__init__(
            nnet=nnet,
            num_classes=num_classes,
            optimizer=optimizer,
            scheduler=scheduler
            )

    def _step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        preds = y_hat.argmax(dim=1)
        return loss, preds, y
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.forward(x)
        return y_hat.argmax(dim=1)

### Usage

In [ ]:
cfg = OmegaConf.load('../config/model/image/resnetx.yaml')
B, C, H, W = 64, 1, 28, 28
x = torch.randn(B, C, H, W)
nnet = instantiate(cfg.nnet, num_classes=10)
y = nnet(x)
print(y.shape)


[15:40:37] INFO - ResNet: init
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used
[15:40:37] WARNING - setting conv bias to False as Batchnorm is used


torch.Size([64, 10])


In [ ]:
summary(nnet, input_size=(B, C, H, W), depth=5)

Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [64, 10]                  --
├─Sequential: 1-1                                  [64, 10]                  --
│    └─ConvLayer: 2-1                              [64, 8, 14, 14]           --
│    │    └─Sequential: 3-1                        [64, 8, 14, 14]           --
│    │    │    └─Conv2d: 4-1                       [64, 8, 14, 14]           72
│    │    │    └─BatchNorm2d: 4-2                  [64, 8, 14, 14]           16
│    │    │    └─ReLU: 4-3                         [64, 8, 14, 14]           --
│    └─ResBlock: 2-2                               [64, 8, 14, 14]           --
│    │    └─Sequential: 3-2                        [64, 8, 14, 14]           --
│    │    │    └─ConvLayer: 4-4                    [64, 8, 14, 14]           --
│    │    │    │    └─Sequential: 5-1              [64, 8, 14, 14]           592
│    │    │    └─ConvLayer: 4-5   

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()